In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from skfuzzy import cmeans
from imblearn.under_sampling import RandomUnderSampler

# # Generate synthetic transaction data (replace with real dataset)
# np.random.seed(42)
# n_samples = 1000
# data = {
#     'amount': np.random.normal(100, 50, n_samples),
#     'time': np.random.uniform(0, 24, n_samples),
#     'location_score': np.random.uniform(0, 1, n_samples),
#     'is_fraud': np.zeros(n_samples)  # 0 for non-fraud, 1 for fraud
# }
# # Simulate 5% fraud cases
# data['is_fraud'][950:] = 1
# data['amount'][950:] *= 5  # Fraudulent transactions have higher amounts
# df = pd.DataFrame(data)
#
# # Preprocess data
# X = df[['amount', 'time', 'location_score']].values
# y = df['is_fraud'].values
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
#
# # Handle class imbalance with undersampling
# rus = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X_scaled, y)


X = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X.csv')
y = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y.csv')['Class']
# Apply Fuzzy C-Means clustering
n_clusters = 3  # Tune based on data
cntr, u, u0, d, jm, p, fpc = cmeans(
    data=X.T, c=n_clusters, m=2, error=0.005, maxiter=1000, init=None
)

# Get membership scores and cluster assignments
membership_scores = u.T
cluster_labels = np.argmax(membership_scores, axis=1)

# Flag suspicious transactions (low membership scores)
threshold = 0.3  # Adjust based on experimentation
is_suspicious = np.max(membership_scores, axis=1) < threshold

# Prepare data for neural network (suspicious cases)
suspicious_indices = np.where(is_suspicious)[0]
X_suspicious = X[suspicious_indices]
y_suspicious = y[suspicious_indices]

# Split data for training neural network
X_train, X_test, y_train, y_test = train_test_split(
    X_suspicious, y_suspicious, test_size=0.2, random_state=42
)

# Train neural network classifier
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# Predict fraud on suspicious cases
y_pred = mlp.predict(X_test)

# Combine results
df_resampled = pd.DataFrame(X, columns=['amount', 'time', 'location_score'])
df_resampled['cluster'] = cluster_labels
df_resampled['is_suspicious'] = is_suspicious
df_resampled['is_fraud'] = y
df_resampled['nn_pred'] = np.nan
df_resampled.loc[suspicious_indices[np.where(np.isin(suspicious_indices, np.where(is_suspicious)[0]))], 'nn_pred'] = mlp.predict(X_suspicious)

# Output results
print("Fraud Detection Results (first 10 rows):")
print(df_resampled.head(10))
print("\nNumber of suspicious cases:", np.sum(is_suspicious))
print("\nNeural Network Accuracy on Suspicious Cases:", mlp.score(X_test, y_test))

# Save results
df_resampled.to_csv('fraud_detection_hybrid_results.csv', index=False)

ValueError: Found input variables with inconsistent numbers of samples: [284506, 0]